<a href="https://colab.research.google.com/github/Saeeeae/Pytorch-Studying/blob/master/gan_dog_kaggle_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"saeeeae","key":"2d19c4dc2ecce44f6afeb0b210d3ad6a"}'}

In [0]:

!kaggle competitions download -c generative-dog-images

403 - Forbidden


In [0]:
import numpy as np
import pandas as pd

import os
print(os.listdir("../input"))

import matplotlib.pyplot as plt
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
#https://www.kaggle.com/jesucristo/gan-introduction

In [0]:
batch_size = 32

# 64x64 images
transform = transforms.Compose([transforms.Resize(64),
                                transforms.CenterCrop(64),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train_data = datasets.ImageFolder('../input/all-dogs/', transform = transform)
train_loader = troch.utils.data.DataLoader(train_data, shuffle=True, batch_szie=batch_size)

imgs, label = next(iter(train_loader))
imgs = imgs.numpy().transpose(0,2,3,1)

In [0]:
 # ConvTranspose2d(a, b, c, d, e)라고 했을 때 각각의 인자:
 # a는 입력으로 들어오는 채널의 수
 # b는 만들어지는 결과값의 채널 수
 # c는 커널의 크기, 즉 Convolution 연산을 수행하는 필터의 크기
 # d는 stride
 # e는 padding

class Generator(nn.Module):
  def __init__(self, nz, nfeats, nchannels):
    super(Genetator, self).__init__()

    # input is Z, going into a convolution
    self.conv1 = nn.ConvTranspose2d(nz, nfeats*8 ,4 ,1, 0, bias=False)
    self.bn1 = nn.BatchNorm2d(nfeats*8)
    #state size (nfeats*8)*4*4

    self.conv2 = nn.ConvTranspose2d(nfeats*8, nfeats*8,4,2,1,bias=False)
    self.bn2 = nn.BatchNorm2d(nfeats*8)
    #state size (nfeats*8)*8*8

    self.conv3 = nn.ConvTranspose2d(nfeats*8,nfeats*4,4,2,1,bias=False)
    self.bn3 = nn.BatchNorm2d(nfeats*4)
    #state size (nfeats*4)*16*16

    self.conv4 = nn.ConvTranspose2d(nfeats*4,nfeats*2,4,2,1,bias=False)
    self.bn4 = nn.BatchNorm2d(nfeats*2)
    #state size (nfeats*2)*32*32

    self.conv5 = nn.ConvTranspose2d(nfeats*2,nfeats,4,2,1,bias=False)
    self.bn5 = nn.BatchNorm2d(nfeats)
    #state size (nfeats)*64*64

    self.conv6 = nn.ConvTranspose2d(nfeats, nchannels,3,1,1,bias=False)
    #state size (nchannels)*64*64

  def forward(self, x):
    x = F.leaky_relu(self.bn1(self.conv1(x)))
    x = F.leaky_relu(self.bn2(self.conv2(x)))
    x = F.leaky_relu(self.bn3(self.conv3(x)))
    x = F.leaky_relu(self.bn4(self.conv4(x)))
    x = F.leaky_relu(self.bn5(self.conv5(x)))
    x = torch.tanh(self.conv6(x))

    return x

In [0]:
class Discriminator(nn.Module):
  def __init__(self, nchannels, nfeats):
    super(Discriminator, self).__init__()

    # input is (nchannels)*64*64
    self.conv1 = nn.Conv2d(nchannels, nfeats, 4, 2, 1, bias=False)
    # state size (nfeats)*32*32

    self.conv2 = nn.Conv2d(nfeats, nfeats*2, 4, 2 ,1, bias=False)
    self.bn2 = nn.BatchNorm2d(nfeats*2)
    # state size (nfeats*2)*16*16

    self.conv3 = nn.Conv2d(nfeats*2, nfeats*4, 4, 2, 1, bias=False)
    self.bn3 = nn.BatchNorm2d(nfeats*4)
    # state size (nfeats*4)*8*8

    self.conv4 = nn.Conv2d(nfeats*4, nfeats*8, 4, 2, 1, bias=False)
    self.bn4 = nn.BatchNorm2d(nfeats*8)
    # state size (nfeats*8)*4*4

    self.conv5 = nn.Conv2d(nfeats*8, 1, 4, 1, 0, bais=False)
    # state size 1*1*1

    def forward(self, x):
      x = F.leaky_relu(self.conv1(x), 0.2)
      x = F.leaky_relu(self.bn2(self.conv2(x)), 0.2)
      x = F.leaky_relu(self.bn3(self.conv3(x)), 0.2)
      x = F.leaky_relu(self.bn4(self.conv4(x)), 0.2)
      x = F.leaky_relu(self.conv5(x))

      return x.view(-1,1)

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

lr = 0.0003
beta1 = 0.5

netG = Generator(100, 32, 3).to(device)
netD = Discriminator(3, 48).to(device)

criterion = nn.BCELoss()

optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

nz = 100
fixed_noise = torch.randn(25, nz, 1, 1, device=device)

real_label = 0.9
fake_label = 0
batch_size = train_loader.batch_size

In [0]:
### training here
epochs = 2

step = 0

for epoch in range(epochs):
  for ii, (real_images, train_labels) in enumerate(train_loader):
    ############################
    # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
    ###########################
    # train with real
    netD.zero_grad()
    real_images = real_images.to(device)
    batch_size = real_images.size(0)
    labels = torch.full((batch_size, 1), real_label, device=device)

    output = netD(real_images)
    errD_real = criterion(output, labels)
    errD_real.backward()
    D_x = output.mean().item()

    ## train with fake
    noise = torch.randn(batch_size, nz, 1, 1, device=device)
    fake = netG(noise)
    labels.fill_(fake_label)
    output = netD(fake.detach())
    errD_fake = criterion(output, labels)
    errD_fake.backward()
    D_G_z1 = ouput.mean().item()
    errD = errD_real + errD_fake
    optimizerD.step()

    netG.zero_grad()
    labels.fill_(real_label)
    output = netD(fake)
    errG = criterion(output, labels)
    errG.backward()
    D_G_z2 = output.mean().item()
    optimizerG.step()

    if step % 500 == 0:
      print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
        % (epoch + 1, epochs, ii, len(train_loader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
            
      valid_image = netG(fixed_noise)
    step += 1
      